<a href="https://colab.research.google.com/github/sergiolucero/notebooks/blob/main/PrefectBLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 0. illustrate Prefect use on BLC pipeline
# 1. create notebook Github repo + add above
# 2. plan + extend

In [ ]:
!pip install prefect s3fs
import pandas as pd
import prefect

In [60]:
from prefect import Flow, task

@task
def extract_reference_data():
    print("fetching reference data...")
    rdf= pd.read_csv('s3://quantcldata/BIKES/blc_reference.csv')
    return rdf

@task
def extract_live_data():  #airport, radius, ref_data):
    BLC_JSON_URL = 'https://www.bicilascondes.cl/availability_map/getJsonObject'
    ldf = pd.read_json(BLC_JSON_URL)
    return ldf
@task 
def transform(live,ref):
    transformed = []
    for lx, rx in zip(live.iterrows(),ref.iterrows()):   # should JOIN
        #print(lx[1])
        ld, rd = lx[1]['bikes'],rx[1]['mean']
        qd = ld/rd
        transformed.append([ld,rd,qd])
    transformed_data = pd.DataFrame(transformed, columns=['live','ref','fill'])
    transformed_data.to_csv('tld.csv', index=False)

with Flow("BLC-ETL") as flow:
    reference_data = extract_reference_data()
    live_data = extract_live_data()
    transform(live_data, reference_data)

    #load_reference_data(reference_data)
    #load_live_data(transformed_live_data)
    flow.run()

[2022-03-02 16:22:40+0000] INFO - prefect.FlowRunner | Beginning Flow run for 'BLC-ETL'
[2022-03-02 16:22:40+0000] INFO - prefect.TaskRunner | Task 'extract_reference_data': Starting task run...
fetching reference data...
[2022-03-02 16:22:40+0000] INFO - prefect.TaskRunner | Task 'extract_reference_data': Finished task run for task with final state: 'Success'
[2022-03-02 16:22:40+0000] INFO - prefect.TaskRunner | Task 'extract_live_data': Starting task run...
[2022-03-02 16:22:41+0000] INFO - prefect.TaskRunner | Task 'extract_live_data': Finished task run for task with final state: 'Success'
[2022-03-02 16:22:41+0000] INFO - prefect.TaskRunner | Task 'transform': Starting task run...
[2022-03-02 16:22:41+0000] INFO - prefect.TaskRunner | Task 'transform': Finished task run for task with final state: 'Success'
[2022-03-02 16:22:41+0000] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


In [61]:
!head *csv

live,ref,fill
7,5.998901098901099,1.1668803810221653
9,4.816818563764462,1.86845318769206
4,3.969943765755284,1.00757094710106
7,6.810871953978412,1.027768551119378
0,3.867300109883007,0.0
6,3.4966711912610693,1.7159177033846609
5,6.9109947643979055,0.7234848484848485
4,5.951199017516644,0.6721334622193742
9,4.456854760519682,2.0193612948137836


In [36]:
from datetime import datetime
import time
for ix in range(5):
  print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), live_data.run()['slots'].sum())
  time.sleep(10)

2022-03-02 16:13:06 1253
2022-03-02 16:13:16 1253
2022-03-02 16:13:27 1253
2022-03-02 16:13:37 1253
2022-03-02 16:13:48 1253


In [54]:
rdf= pd.read_csv('s3://quantcldata/BIKES/blc_reference.csv')
rdf

,id,min,mean,max,std,lat,lon
0,1,0,5.998901,13,2.560699,-33.411387,-70.602392
1,2,0,4.816819,15,3.064665,-33.414113,-70.601854
2,3,0,3.969944,18,2.632439,-33.413891,-70.592589
3,4,0,6.810872,11,1.936994,-33.415341,-70.603824
4,5,0,3.867300,12,2.087277,-33.416998,-70.601671
...,...,...,...,...,...,...,...
93,96,0,3.374467,5,0.967429,-33.375038,-70.504996
94,97,4,4.000000,4,0.000000,-33.375780,-70.509120
95,98,0,5.481965,7,0.881295,-33.415982,-70.534215
96,99,0,4.514867,5,0.508260,-33.416379,-70.529415


In [18]:
for row in reference_data:
  print(row)

TypeError: ignored